## **FewShotPromptTemplate**

- Few-shot 학습은 언어 모델에 몇 가지 예시를 제공하여 특정 작업을 수행하도록 유도하는 기법.

- Few-shot 학습을 활용함으로써, 언어 모델은 주어진 예제들을 참고하여 더 정확하고 일관된 응답을 생성


### **1. Few-shot 예제 포맷터 생성**

- Few-shot 예제를 포맷팅하기 위한 템플릿을 생성. 

- PromptTemplate 은 질문과 답변을 포함하는 구조 

- 템플릿은 각 예제를 일관된 형식으로 표현할 수 있게 해주어, 모델이 입력과 출력의 패턴을 쉽게 인식.

In [72]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n답변 : {answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='질문: {question}\n답변 : {answer}')

### **2. 예제**

- 모델이 참조 할 수 있는 Q&A 예제 세트

In [73]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


In [74]:
#  ** 연산자는 딕셔너리의 key-value 형태를 풀어서 함수나 메서드에 개별 인자로 전달하는 역할

print(example_prompt.format(**examples[0]))

질문: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
답변 : 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



### **3. FewShotPromptTemplate 생성**


- Few-shot 프롬프트 템플릿을 생성, 새로운 질문에 대한 프롬프트를 생성. 

- FewShotPromptTemplate 은 예제들을 결합하고 새로운 입력을 추가하여 최종 프롬프트를 생성.

In [76]:
from langchain_core.prompts import FewShotPromptTemplate
from langchain_groq import ChatGroq

# FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "질문: {input}",
    input_variables = ["input"],
)
# 새로운 질문에 대한 프롬프트를 생성하고 출력합니다.
# print(prompt.invoke({"input": "화성의 표면이 붉은 이유는 무엇인가요?"}).to_string())

final_prompt = prompt.format(input = "구글의 창립자는 언제 태어났나요?")

llm = ChatGroq(model = 'gemma2-9b-it')

answer = llm.invoke(final_prompt)

print(answer.content)

답변 : 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 구글의 창립자는 누구인가요?
중간 답변: 구글의 창립자는 래리 페이지와 세르게이 브린입니다.
추가 질문: 래리 페이지는 언제 태어났나요?
중간 답변: 래리 페이지는 1973년 3월 26일에 태어났습니다.
추가 질문: 세르게이 브린은 언제 태어났나요?
중간 답변: 세르게이 브린은 1973년 8월 21일에 태어났습니다.
최종 답변은: 래리 페이지는 1973년 3월 26일, 세르게이 브린은 1973년 8월 21일에 태어났습니다. 





### **4. 예제 선택기 사용하기**


- 의미적 유사성을 기반으로 가장 관련성 높은 예제를 선택. 

- `SemanticSimilarityExampleSelector` 는 입력 질문과 가장 유사한 예제를 선택.

In [ ]:
!pip install langchain_chroma

In [85]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# SemanticSimilarityExampleSelector를 초기화합니다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,            # 사용할 예제들
    OpenAIEmbeddings(),  # 임베딩 모델
    Chroma,              # 벡터 저장소
    k=1,                 # 선택할 예제 수
)

# 새로운 질문에 대해 가장 유사한 예제를 선택합니다.
question = "헬렌켈러와 히틀러중 누가 더 오래 살았나요?"
selected_examples = example_selector.select_examples({"question": question})

print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")


입력과 가장 유사한 예제: 헬렌켈러와 히틀러중 누가 더 오래 살았나요?


answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인

question: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?


## **채팅 모델에서 Few-shot 예제 사용하기**

### **5. 고정 예제 사용하기**

In [93]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# 예제 정의 
# 예제 정의
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
]

# 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        
        ('human', "{input}"),
        ('ai', "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 정의
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델과 체인 생성
model = ChatGroq(model="gemma2-9b-it", temperature=0)
chain = final_prompt | model

# 모델에 질문하기
result = chain.invoke({"input": "LLM은 무엇의 약자인가요?"})
print(result.content)

LLM은 **Large Language Model**의 약자입니다. 

즉, 대규모 언어 모델이라는 뜻입니다. 😊 



In [97]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 더 많은 예제 추가
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
    {"input": "피타고라스 정리를 설명해주세요.", "output": "직각삼각형에서 빗변의 제곱은 다른 두 변의 제곱의 합과 같습니다."},
    {"input": "DNA의 기본 구조를 간단히 설명해주세요.", "output": "DNA는 이중 나선 구조를 가진 핵산입니다."},
    {"input": "원주율(π)의 정의는 무엇인가요?", "output": "원의 둘레와 지름의 비율입니다."},
]

# 벡터 저장소 생성
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# 예제 선택기 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
# 모델과 체인 생성
model = ChatGroq(model="gemma2-9b-it", temperature=0)
chain = final_prompt | model

# 모델에 질문하기
result = chain.invoke('태양계에서 가장 큰 행성은 무엇인가요?')
print(result.content)

태양계에서 가장 큰 행성은 목성입니다. 🪐  


목성은 지구의 약 11배나 큰 규모를 자랑합니다. 



In [95]:
few_shot_prompt

FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001BA5D4E8D50>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None), input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))